In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git config --global user.name "alistvt"
!git config --global user.email "alistvt@gmail.com"

%cd /content/drive/MyDrive/multidoc-conv-qa/src/retriever

/content/drive/MyDrive/multidoc-conv-qa/src/retriever


In [3]:
%cd ..

e:\Projects\nlp\multidoc-conv-qa\src\retriever


In [ ]:
!pip install -r ../../requirements.txt --quiet
!pip install --quiet transformers datasets accelerate

In [5]:
from tqdm import tqdm
import numpy as np


In [6]:
import json
with open('../../dataset/multidoc2dial/v1.0/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

In [7]:
titles = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        titles.append(doc_idx2)

In [8]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())
        title_to_domain[doc_idx2] = doc_idx1

In [9]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('punkt')

def my_simple_tokenizer(sent):
    # sent = "this is a foo bar! bar black sheep."
    stop = set(stopwords.words('english') + list(string.punctuation))
    try:
        new_words = [i for i in word_tokenize(sent.lower()) if i not in stop and len(i)>1]
    except:
        print(sent)
    return new_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Calculating the IDF for each token

In [10]:
words_idf_file = 'IDFs.pkl'
N_doc = len(doc_texts_train)

if True:
# if not os.path.exists(words_idf_file):
    # First getting all distinct words in all documents
    words = set()
    doc_texts_train_tokenized = []
    for doc in tqdm(doc_texts_train, desc="getting all words from documents"):
        doc = doc.replace("#", "")
        tokenized_doc = my_simple_tokenizer(doc)
        # tokenized_doc = [s.lower() for s in tokenizer.tokenize(doc)]
        doc_texts_train_tokenized.append(tokenized_doc) 
        words = set(tokenized_doc).union(words)

    # calculating each word IDF
    words2IDF = {}
    for word in tqdm(words, desc="calculating words IDF scores"):
        n_word = 0
        for doc in doc_texts_train_tokenized:
            if word in doc:
                n_word += 1
        words2IDF[word] = np.log(N_doc / (n_word + 1))

    # with open(words_idf_file, 'wb') as f:
        # pickle.dump(words2IDF, f)

else:
    with open(words_idf_file, 'rb') as f:
        words2IDF = pickle.load(f)

calculating words IDF scores: 100%|██████████| 8912/8912 [00:27<00:00, 327.27it/s]


In [ ]:
print(sorted(words2IDF.keys()))

In [15]:
def calc_idf_score(sentence):
    """
    Calculate the mean idf score for given sentence.
    (used to understand the contribution of the knowledge of each question
    questions with high frequent words are meaningless and we can ignore them
    roughly, which is done by this score.)

    :param sentence: input sentence
    :type sentence: str
    :return: mean idf score of sentence token
    """
    tokenzied_sentence = [s.lower() for s in my_simple_tokenizer(sentence)]
    # tokenzied_sentence = [s.lower() for s in tokenizer.tokenize(sentence)]
    score = 0
    for token in tokenzied_sentence:
        if token in words2IDF:
            score += words2IDF[token]
        else:
            score += np.log(N_doc)
    return score / len(tokenzied_sentence) if len(tokenzied_sentence) else 0

In [16]:
def calc_insideness_score(sentence, doc):
    doc = doc.lower()
    sentence = sentence.lower()
    sentence_tokens = my_simple_tokenizer(sentence)
    doc_tokens = my_simple_tokenizer(doc)
    score = 0
    for doc_token in doc_tokens:
        for sentence_token in sentence_tokens:
            if doc_token in sentence_token:
                score += 1
            if sentence_token in doc_token:
                score += 1
    return score / (2 * len(doc_tokens))

In [17]:
calc_insideness_score("hello world", "HELLO WORLDs")

0.75

In [18]:
def combine_sentences(s1, s2):
    separation_token = " <SEP> "
    return s1 + separation_token + s2


def construct_followup_dataset(filepath):
    import json
    with open(filepath, 'r') as f:
        multidoc2dial_dial_train = json.load(f)
    
    historys = []
    questions = []
    combined = []
    labels = []
    prev_docs = []
    current_docs = []
    prev_answers = []

    for domain in multidoc2dial_dial_train['dial_data']:
        for dial in multidoc2dial_dial_train['dial_data'][domain]:
            prev_doc = ''
            prev_question = ''
            prev_answer = ''
            for turn in dial['turns']:
                if turn['role'] == "user":
                    current_question = turn['utterance']
                    historys.append(prev_question)
                    questions.append(current_question)
                    
                    combined.append(combine_sentences(prev_question, current_question))

                    current_doc = turn['references'][0]['doc_id']
                    labels.append(int(current_doc==prev_doc))

                    prev_docs.append(prev_doc)
                    current_docs.append(current_doc)
                    prev_answers.append(prev_answer)

                    prev_doc, prev_question = current_doc, current_question
                else:
                    prev_answer = turn['utterance']
                    
    return historys, questions, combined, labels, prev_docs, current_docs, prev_answers

In [20]:
import pandas as pd

train_history, train_questions, train_combined, train_labels, train_prev_docs, train_current_docs, train_prev_answers = construct_followup_dataset('../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_train.json')
test_history, test_questions, test_combined, test_labels, test_prev_docs, test_current_docs, test_prev_answers = construct_followup_dataset('../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_validation.json')

train_dict_dataset = {"history":train_history, "question": train_questions, "combined": train_combined, "followup": train_labels, "prev_doc": train_prev_docs, "current_doc": train_current_docs, "prev_answer": train_prev_answers}
test_dict_dataset = {"history":test_history, "question": test_questions, "combined": test_combined, "followup": test_labels, "prev_doc": test_prev_docs, "current_doc": test_current_docs, "prev_answer": test_prev_answers}

train_df = pd.DataFrame(train_dict_dataset)
test_df = pd.DataFrame(test_dict_dataset)

In [19]:
max([len(x["combined"].split()) for _, x in test_df.iterrows()])

NameError: name 'test_df' is not defined

#### TF-IDF Transformation Matrix Fitting

In [31]:
doc_texts_train = []
doc_ids = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_ids.append(doc_idx2)
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(strip_accents=None,
                                 analyzer='char',
                                 ngram_range=(3, 8),
                                 norm='l2',
                                 use_idf=True,
                                 smooth_idf=True)
tfidf_wm = tfidfVectorizer.fit_transform(doc_texts_train)

In [24]:
# import pickle
# with open('tfidfVectorizer.pkl', 'wb') as f:
#     pickle.dump(tfidfVectorizer, f)

# with open('tfidf_wm.pkl', 'wb') as f:
#     pickle.dump(tfidf_wm, f)

In [25]:
len(tfidfVectorizer.get_feature_names_out())

1046177

In [39]:
print(train_df.iloc[0])
query = train_df.iloc[2]['question']

insideness = [(calc_insideness_score(query, doc), i) for i,doc in enumerate(doc_texts_train)]

history                                                         
question       Hello, I forgot o update my address, can you h...
combined        <SEP> Hello, I forgot o update my address, ca...
followup                                                       0
prev_doc                                                        
current_doc         Top 5 DMV Mistakes and How to Avoid Them#3_0
prev_answer                                                     
Name: 0, dtype: object


In [37]:
doc_ids.index('Top 5 DMV Mistakes and How to Avoid Them#3_0')

250

In [40]:
sorted(insideness, key=lambda x: x[0], reverse=True)

[(0.11394557823129252, 310),
 (0.10971223021582734, 299),
 (0.10507246376811594, 356),
 (0.10060975609756098, 276),
 (0.09464285714285714, 200),
 (0.09426987060998152, 388),
 (0.09297052154195011, 341),
 (0.09269662921348315, 363),
 (0.09243697478991597, 260),
 (0.09217877094972067, 304),
 (0.09195402298850575, 338),
 (0.09183673469387756, 290),
 (0.09027777777777778, 307),
 (0.0889423076923077, 298),
 (0.08710801393728224, 392),
 (0.08222222222222222, 359),
 (0.08196721311475409, 302),
 (0.08076009501187649, 250),
 (0.08, 390),
 (0.07886435331230283, 252),
 (0.07828282828282829, 332),
 (0.0781010719754977, 305),
 (0.07802874743326489, 362),
 (0.07624113475177305, 320),
 (0.07608695652173914, 271),
 (0.07288135593220339, 324),
 (0.06735751295336788, 361),
 (0.06666666666666667, 283),
 (0.06584362139917696, 309),
 (0.06533333333333333, 335),
 (0.06521739130434782, 247),
 (0.060526315789473685, 297),
 (0.06030534351145038, 249),
 (0.05982905982905983, 364),
 (0.05952380952380952, 349),
 

#### DR. TEIT

the input is consisted of a list of queries, which is the current question and its history turns.
for each of the questions, we compute two similarity score for each of our documents, one of them is based on the pretrained LM and the other on is based on character level matching. Both of these scores will be weighted by a coefficient which is the `idf_score` of the query, defining how much meaning does the query contain. Then these scores will be summed up in a convex manner and the final matching score with all documents is computed. We return the result by sorting these scores.

In [34]:
def predict_DR_TEIT(data, k=1, alpha=10):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    queries = [data['question'], data['prev_answer'], data['history']]
    idf_score = np.array(list([0.0] * len(doc_texts_train)))
    tfidf_score = np.array(list([0.0] * len(doc_texts_train)))

    coef_sum = 0.0
    for i, query in enumerate(queries):
        query_sim = np.array([calc_insideness_score(query, doc) for doc in doc_texts_train])
        coef = calc_idf_score(query) * (2**(-i))
        coef_sum += coef

        idf_score += coef * query_sim
        tfidf_score += coef * np.squeeze(np.asarray(tfidf_wm @ tfidfVectorizer.transform([query]).todense().T))

    scores = (idf_score + alpha * tfidf_score) / coef_sum
    best_k_idx = scores.argsort()[::-1][:k]
    scores = scores[best_k_idx]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    return predictions

In [1]:
cache_insideness = {}

In [49]:
for index, data in tqdm(test_df.iterrows()):
    queries = [data['question'], data['prev_answer'], data['history']]
    for q in queries:
        cache_insideness[q] = [calc_insideness_score(q, doc) for doc in doc_texts_train]

9it [01:27,  9.75s/it]


KeyboardInterrupt: 

# Test
In the test dataset we just picked ones with **user** turn.

In [40]:
def test_loop(df, predictor):
    prec_at_50 = 0
    prec_at_10 = 0
    prec_at_5 = 0
    prec_at_1 = 0
    ranks = []
    for index, data in tqdm(df.iterrows()):
        predictions = predictor(data, k=500)
        actual_doc = data['current_doc']
        ranks.append(1 / (predictions.index(actual_doc) + 1))
        if actual_doc == predictions[0]:
            prec_at_1 += 1
        if actual_doc in predictions[:5]:
            prec_at_5 += 1
        if actual_doc in predictions[:10]:
            prec_at_10 += 1
        if actual_doc in predictions[:50]:
            prec_at_50 += 1

        if index % 50 == 49:
            print(f"""
                MRR: mean={np.array(ranks).mean()}, var={np.array(ranks).var()}
                Prec@(1) = {prec_at_1 / index}
                Prec@(5) = {prec_at_5 / index}
                Prec@(10) = {prec_at_10 / index}
                Prec@(50) = {prec_at_50 / index}
                NUMBER_OF_SAMPLES = {index}
            """)

    return f"""
        MRR: mean={np.array(ranks).mean()}, var={np.array(ranks).var()}
        Prec@(1) = {prec_at_1 / index}
        Prec@(5) = {prec_at_5 / index}
        Prec@(10) = {prec_at_10 / index}
        Prec@(50) = {prec_at_50 / index}
        NUMBER_OF_SAMPLES = {index}
    """

In [47]:
print(test_loop(train_df, predict_DR_TEIT))
# print(test_loop(train_df, predict_FUDNet_DR_TEIT))

50it [05:00,  5.95s/it]


                MRR: mean=0.6340838781101938, var=0.15844104038255055
                Prec@(1) = 0.5306122448979592
                Prec@(5) = 0.7346938775510204
                Prec@(10) = 0.9183673469387755
                Prec@(50) = 1.0204081632653061
                NUMBER_OF_SAMPLES = 49
            


54it [05:29,  6.10s/it]


KeyboardInterrupt: 

# Results

At last we have resutls as follows:


| Method | @1 | @5 | @10 | @50 | @100 | MRR (mean, var) |
|:------:|:------:|:------:|:-------:|:-------:|:--------:|:---:|
| IDF - vanilla | 13% | 30% | 39% | 64% | 83% | (0.22, 0.11) |
| IDF - power-order | 15% | 31% | 41% | 65% | 83% | (0.23, 0.12) |
| IDF - power-order (softmax) | 10.7% | 23% | 31% | 57.6% | 78% | (0.18, 0.09) |
| IDF - self-attention | 13.9% | 29% | 38% | 62% | 82% | (0.22, 0.11) |
| DR. TEIT | 61.6% | 86% | 91% | 96% | 98% | (0.72, 0.13) |
| DR. TEIT (val) | 40.0% | 69.25% | 92.4% | 99% | 98% | (0.53, 0.16) |
| FUDNet + DR. TEIT | 67% | 87% | 91% | 96% | 99% | (0.76, 0.12) |
| FUDNet + DR. TEIT (val) | 48.2% | 75.48% | 82% | 93% | 99% | (0.60, 0.16) |

MRR: mean=0.7605851824924286, var=0.12850709866222307
                Prec@(1) = 0.6706096451319381
                Prec@(5) = 0.8698817106460418
                Prec@(10) = 0.910828025477707
                Prec@(50) = 0.9681528662420382
                NUMBER_OF_SAMPLES = 1099